## Rating Products & Sorting Reviews in Amazon Data


#### In this project I tried to apply product rating and review sorting methods to Amazon Data

#### 1- Our first goal is to calculate the rating of a product and understand its trend. 

#### 2 -Our second goal is to determine 5 reviews out of almost 5000 reviews to list under the product

##### Link of data is as follows. But we will use a sub_set of data 
##### http://jmcauley.ucsd.edu/data/amazon/links.html

#### Informations about data:

##### reviewerID - ID of the reviewer, e.g. A2SUAM1J3GNN3B
##### asin - ID of the product, e.g. 0000013714
##### reviewerName - name of the reviewer
##### helpful - helpfulness rating of the review, e.g. 2/3
##### reviewText - text of the review
##### overall - rating of the product
##### summary - summary of the review
##### unixReviewTime - time of the review (unix time)
##### reviewTime - time of the review (raw)

### Imports

In [1]:
import pandas as pd
import scipy.stats as st
import math
import gzip

##### We will take a look only the product that has received the highest number of reviews.
##### There for the fallowing csv file consist of only one products information

### Rating Product

In [2]:
df= pd.read_csv("df_sub.csv")
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [3]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A3SBTW3WS4IQSN,B007WTAJTO,NaN,"[0, 0]",No issues.,4.00,Four Stars,1406073600,"07 23, 2014"
1,A18K1ODH1I2MVB,B007WTAJTO,0mie,"[0, 0]","Purchased this for my device, it worked as adv...",5.00,MOAR SPACE!!!,1382659200,"10 25, 2013"
2,A2FII3I2MBMUIA,B007WTAJTO,1K3,"[0, 0]",it works as expected. I should have sprung for...,4.00,nothing to really say....,1356220800,"12 23, 2012"
3,A3H99DFEG68SR,B007WTAJTO,1m2,"[0, 0]",This think has worked out great.Had a diff. br...,5.00,Great buy at this price!!! *** UPDATE,1384992000,"11 21, 2013"
4,A375ZM4U047O79,B007WTAJTO,2&amp;1/2Men,"[0, 0]","Bought it with Retail Packaging, arrived legit...",5.00,best deal around,1373673600,"07 13, 2013"


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4915 entries, 0 to 4914
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   reviewerID      4915 non-null   object 
 1   asin            4915 non-null   object 
 2   reviewerName    4914 non-null   object 
 3   helpful         4915 non-null   object 
 4   reviewText      4914 non-null   object 
 5   overall         4915 non-null   float64
 6   summary         4915 non-null   object 
 7   unixReviewTime  4915 non-null   int64  
 8   reviewTime      4915 non-null   object 
dtypes: float64(1), int64(1), object(7)
memory usage: 345.7+ KB


##### As you can see our review time variable is in object type lets convert it to datetime

In [5]:
df['reviewTime'] = pd.to_datetime(df['reviewTime'], dayfirst=True)

##### Lets see what is the mean of overall rating of this product

In [6]:
df["overall"].mean()

4.587589013224822

##### Lets see how the product's rating has changed over time and understand its trend 

##### To do that lets create a variable "day_diff" to differentiate old ratings and new ratings

##### As this data is from 2014, lets define the current date as some day in 2014 to calculate ratings age in days

In [7]:
current_date = pd.to_datetime('2014-12-08 0:0:0')
df["day_diff"] = (current_date - df['reviewTime']).dt.days

##### Lets see what it is look like

In [8]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff
0,A3SBTW3WS4IQSN,B007WTAJTO,NaN,"[0, 0]",No issues.,4.00,Four Stars,1406073600,2014-07-23,138
1,A18K1ODH1I2MVB,B007WTAJTO,0mie,"[0, 0]","Purchased this for my device, it worked as adv...",5.00,MOAR SPACE!!!,1382659200,2013-10-25,409
2,A2FII3I2MBMUIA,B007WTAJTO,1K3,"[0, 0]",it works as expected. I should have sprung for...,4.00,nothing to really say....,1356220800,2012-12-23,715
3,A3H99DFEG68SR,B007WTAJTO,1m2,"[0, 0]",This think has worked out great.Had a diff. br...,5.00,Great buy at this price!!! *** UPDATE,1384992000,2013-11-21,382
4,A375ZM4U047O79,B007WTAJTO,2&amp;1/2Men,"[0, 0]","Bought it with Retail Packaging, arrived legit...",5.00,best deal around,1373673600,2013-07-13,513


##### Now lets use quantile values to set treshold values to score the age of rating.



In [9]:
a = df["day_diff"].quantile(0.25)
b = df["day_diff"].quantile(0.50)
c = df["day_diff"].quantile(0.75)

##### Now lets use these quantile values to set our own weighted rating score which is in favor of recent ratings

In [10]:
df.loc[df["day_diff"] <= a, "overall"].mean() * 28 / 100 + \
    df.loc[(df["day_diff"] > a) & (df["day_diff"] <= b), "overall"].mean() * 26 / 100 + \
    df.loc[(df["day_diff"] > b) & (df["day_diff"] <= c), "overall"].mean() * 24 / 100 + \
    df.loc[(df["day_diff"] > c), "overall"].mean() * 22 / 100

4.595593165128118

##### Well as you can see our weighted rating resulted better than the mean we calculated above. This means that the rating of our product is in a trend to increase

### Sorting Reviews

##### Now let say our goal is to determine 5 user reviews to list under the product
##### Which reviews deserved to be listed under the product?
##### Lets find out.


In [11]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff
0,A3SBTW3WS4IQSN,B007WTAJTO,NaN,"[0, 0]",No issues.,4.00,Four Stars,1406073600,2014-07-23,138
1,A18K1ODH1I2MVB,B007WTAJTO,0mie,"[0, 0]","Purchased this for my device, it worked as adv...",5.00,MOAR SPACE!!!,1382659200,2013-10-25,409
2,A2FII3I2MBMUIA,B007WTAJTO,1K3,"[0, 0]",it works as expected. I should have sprung for...,4.00,nothing to really say....,1356220800,2012-12-23,715
3,A3H99DFEG68SR,B007WTAJTO,1m2,"[0, 0]",This think has worked out great.Had a diff. br...,5.00,Great buy at this price!!! *** UPDATE,1384992000,2013-11-21,382
4,A375ZM4U047O79,B007WTAJTO,2&amp;1/2Men,"[0, 0]","Bought it with Retail Packaging, arrived legit...",5.00,best deal around,1373673600,2013-07-13,513


##### We hava a variabale "helpful" as you can see above lets make use of it. The first value in helpful variable represents how many times a review voted as helpful and the second value represents the total vote recived for rach reviews
##### As we are not going to use some variables lets get rid of them 
##### Also lets sort the values according to their total vote given hepful or not

In [12]:
df = df[["helpful","reviewText","overall"]]
df['helpful_yes'] = df[['helpful']].applymap(lambda x : x.split(', ')[0].strip('[')).astype(int)
df['total_vote'] = df[['helpful']].applymap(lambda x : x.split(', ')[1].strip(']')).astype(int)
df['helpful_no'] = df['total_vote'] - df['helpful_yes']
df = df.sort_values("total_vote", ascending=False )
df.head()

,helpful,reviewText,overall,helpful_yes,total_vote,helpful_no
2031,"[1952, 2020]",[[ UPDATE - 6/19/2014 ]]So my lovely wife boug...,5.00,1952,2020,68
4212,"[1568, 1694]",NOTE: please read the last update (scroll to ...,1.00,1568,1694,126
3449,"[1428, 1505]",I have tested dozens of SDHC and micro-SDHC ca...,5.00,1428,1505,77
317,"[422, 495]","If your card gets hot enough to be painful, it...",1.00,422,495,73
2909,"[53, 236]",I know armed with this in my Android tablet an...,4.00,53,236,183


##### As you can see above we first selected "helpful","reviewText","overall" variablesand then we created 3 variables 'helpful_yes','helpful_no' and 'total_vote' out of our good old variable "helpful"

##### Now lets define a simple function to calculate the average helpfulness score for each review

In [13]:
def score_average(pos, neg):
    if pos + neg == 0:
        return 0
    return pos / (pos + neg)

In [14]:
df["average_score"] = df.apply(lambda x: score_average(x["helpful_yes"], x["helpful_no"]), axis=1)
df = df.sort_values("average_score", ascending=False )
df.head()

,helpful,reviewText,overall,helpful_yes,total_vote,helpful_no,average_score
1989,"[1, 1]",Perfect. Took card put old 32gb card and tran...,5.00,1,1,0,1.00
4239,"[1, 1]",I got this product exactly 1 month and 3 days ...,1.00,1,1,0,1.00
4258,"[1, 1]",I ordered these cards at a great price during ...,2.00,1,1,0,1.00
1825,"[1, 1]",I purchased this memory card to use to expand ...,5.00,1,1,0,1.00
2008,"[1, 1]",This is the second one of these chips that I r...,5.00,1,1,0,1.00


##### As you can see we created a score for each review according to given votes wheter helpful or not and listed them according to their scores. 
##### However as you can see this method has a weakness and favoring the reviews received 0 "not helpful" vote without considering how many votes received in total.
##### For example if we use this method, while sorting the reviews under the product the review 1 below will be listed above the Review2 which is a catastrophic mistake companies fall into even nowadays

##### Review1: 1 helpful, 0 not helpful
##### Review2: 99 helpful, 1 not helpful

##### That is why we have to do better than that. Luckyly good old uncle Wilson has a solution. Lets define another function in wich we will take all the parameters which are "helpful, not helpful and total vote"


In [15]:
def wilson_lower(pos, neg, confidence=0.95):
    
    """
    
    Function to provide lower bound of wilson score
    :param pos: No of positive ratings
    :param neg: Total number of ratings
    :param confidence: Confidence interval, by default is 95 %
    :return: Wilson Lower bound score
    
    """
    n = pos + neg
    if n == 0:
        return 0
    z = st.norm.ppf(1 - (1 - confidence) / 2)
    phat = 1.0 * pos / n
    return (phat + z * z / (2 * n) - z * math.sqrt((phat * (1 - phat) + z * z / (4 * n)) / n)) / (1 + z * z / n)

In [16]:
df["wilson_lower"] = df.apply(lambda x: wilson_lower(x["helpful_yes"], x["helpful_no"]), axis=1)
df = df.sort_values("wilson_lower", ascending=False )
df.head()

,helpful,reviewText,overall,helpful_yes,total_vote,helpful_no,average_score,wilson_lower
2031,"[1952, 2020]",[[ UPDATE - 6/19/2014 ]]So my lovely wife boug...,5.00,1952,2020,68,0.97,0.96
3449,"[1428, 1505]",I have tested dozens of SDHC and micro-SDHC ca...,5.00,1428,1505,77,0.95,0.94
4212,"[1568, 1694]",NOTE: please read the last update (scroll to ...,1.00,1568,1694,126,0.93,0.91
317,"[422, 495]","If your card gets hot enough to be painful, it...",1.00,422,495,73,0.85,0.82
4672,"[45, 49]",Sandisk announcement of the first 128GB micro ...,5.00,45,49,4,0.92,0.81


##### As you can see we created a wilson lower bound score for each review according to given votes wheter helpful or not and listed them according to their WLB scores. 
##### Note that by using this method we took all the variables regarding helpfulness of a review into consideration and succesfully listed them down 

### Conclusion

##### In this project our goal was to calculate the rating of a product and understand its trend. To do that we created a function which calculates the rating of a product by favoring the recent votes given.

##### Our other goal was to determine 5 reviews out of almost 5000 reviews to list under the product. To do that we made use of Wilson Lower Bound and listed our reviews according to their helpfulness score.

